In [1]:
import numpy as np
import optuna
import pandas as pd
import tensorflow as tf

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from keras.src import Model, Input, optimizers
from keras.src.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.src.layers import Activation, Dense
from keras.src.legacy.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from typing import List

# Importing the TCN layer from a custom module
from tcn_layer import TCN, adjust_dilations

2025-06-17 15:06:36.579829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750140396.590950 2658772 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750140396.594371 2658772 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750140396.604296 2658772 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750140396.604305 2658772 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750140396.604307 2658772 computation_placer.cc:177] computation placer alr

In [2]:
def normalize_data(data_df: pd.DataFrame, feature_ranges: dict) -> pd.DataFrame:
    ''' Normalize the data in the DataFrame based on predefined feature ranges.'''
    for col, (min_val, max_val) in feature_ranges.items():
        if col in data_df.columns:
            data_df[col] = (data_df[col] - min_val) / (max_val - min_val)

    return data_df

def inverse_normalize(data, feature_ranges):
    for col, (min_val, max_val) in feature_ranges.items():
        if col in data.columns:
            data[col] = data[col] * (max_val - min_val) + min_val
            
    return data
    
def generate_sequences(X, y, look_back):
    gen = TimeseriesGenerator(X, y, length=look_back, batch_size=len(X))

    return gen[0] if len(gen) > 0 else (None, None)

def data_preprocessor(data_df: pd.DataFrame, input_cols: List[str], output_cols: List[str], power_cols: List[str], feature_ranges: dict, look_back: int):
    # 1. Remove unnecessary columns
    data_df = data_df.drop(columns=['datetime'] + power_cols)
    # 2. Normalize the data
    data_df = normalize_data(data_df, feature_ranges)
    # 3. Generate sequences for dataset
    sequence_data = generate_sequences(data_df[input_cols].values, data_df[output_cols].values, look_back=look_back)
    # 4. Split the data into training, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(sequence_data[0], sequence_data[1], test_size=0.1, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    return X_train, y_train, X_val, y_val, X_test, y_test

def compiled_tcn(num_infeat: int, num_outfeat: int, max_len: int, 
                nb_filters: int, kernel_size: int, dilations: List[int], nb_stacks: int, dropout_rate: float = 0.05, activation: str = 'relu', learning_rate: float = 0.002,
                use_layer_norm: bool = True, use_batch_norm: bool = False, padding: str = 'causal', use_skip_connections: bool = True, 
                name: str = 'tcn', kernel_initializer: str = 'he_normal'):

    dilations = adjust_dilations(dilations)

    input_layer = Input(shape=(max_len, num_infeat))

    x = TCN(nb_filters, kernel_size, nb_stacks, dilations, padding,
            use_skip_connections, dropout_rate, 
            activation, kernel_initializer, use_batch_norm, use_layer_norm, return_sequences=False,
            name=name+'1')(input_layer)
    
    # x = TCN(nb_filters, kernel_size, nb_stacks, dilations, padding,
    #         use_skip_connections, dropout_rate, 
    #         activation, kernel_initializer, use_batch_norm, use_layer_norm, return_sequences=False,
    #         name=name+'2')(x)

    print('x.shape=', x.shape)

    opter = optimizers.Adam(learning_rate=learning_rate, clipnorm=1.)

    x = Dense(num_outfeat)(x)
    x = Activation('linear')(x)
    output_layer = x
    model = Model(input_layer, output_layer)
    model.compile(opter, loss='mean_squared_error')
    print('model.x = {}'.format(input_layer.shape))
    print('model.y = {}'.format(output_layer.shape))

    return model

def model_trainer(input_cols, output_cols, look_back, epochs, batch_size, X_train, y_train, X_val, y_val, param_grid):
    best_model_holder = {'model': None, 'score': float('inf')}
    history_holder = {'history': None}

    def objective(trial):
        nb_filters = trial.suggest_categorical('nb_filters', param_grid['nb_filters'])
        kernel_size = trial.suggest_categorical('kernel_size', param_grid['kernel_size'])
        dropout_rate = trial.suggest_float('dropout', 0.2, 0.5)
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)

        # Dynamic video memory management
        gpus = tf.config.experimental.list_physical_devices('GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        model = compiled_tcn(num_infeat=len(input_cols), num_outfeat=len(output_cols), max_len=look_back,
                        nb_filters=nb_filters, kernel_size=kernel_size, dilations=[1, 1, 1, 1, 1, 1], nb_stacks=1,
                        dropout_rate=0.05, activation='relu', learning_rate=0.002,
                        use_layer_norm=True, use_batch_norm=False, padding='causal',
                        use_skip_connections=True)

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, reduce_lr])

        min_val_loss = min(history.history['val_loss'])
        if min_val_loss < best_model_holder['score']:
            best_model_holder['score'] = min_val_loss
            best_model_holder['model'] = model
            history_holder['history'] = history

        return min_val_loss
    
    # Start Optuna Study
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=3, show_progress_bar=True)
    print("Best hyperparameters:", study.best_params)
    print("Best validation MSE:", study.best_value)

    return best_model_holder['model'], history_holder['history']

def model_evaluator(model, X_test, y_test, output_cols, feature_ranges):
    y_pred = model.predict(X_test)

    y_test_df = pd.DataFrame(y_test, columns=output_cols)
    y_pred_df = pd.DataFrame(y_pred, columns=output_cols)
    y_pred_inv = inverse_normalize(y_pred_df, feature_ranges)
    y_test_inv = inverse_normalize(y_test_df, feature_ranges)

    polts = draw_evaluation(output_cols, y_test_inv, y_pred_inv)

    mae_in_temp = mean_absolute_error(y_test_inv['in_temp'], y_pred_inv['in_temp'])
    mae_in_humi = mean_absolute_error(y_test_inv['in_humi'], y_pred_inv['in_humi'])
    mae_in_co2 = mean_absolute_error(y_test_inv['in_co2'], y_pred_inv['in_co2'])
    
    mse_in_temp = mean_squared_error(y_test_inv['in_temp'], y_pred_inv['in_temp'])
    mse_in_humi = mean_squared_error(y_test_inv['in_humi'], y_pred_inv['in_humi'])
    mse_in_co2 = mean_squared_error(y_test_inv['in_co2'], y_pred_inv['in_co2'])

    rmse_in_temp = np.sqrt(mse_in_temp)
    rmse_in_humi = np.sqrt(mse_in_humi)
    rmse_in_co2 = np.sqrt(mse_in_co2)

    return polts, mae_in_temp, mae_in_humi, mae_in_co2, mse_in_temp, mse_in_humi, mse_in_co2, rmse_in_temp, rmse_in_humi, rmse_in_co2

def draw_traing_history(history):
    output_notebook()
    epochs = list(range(1, len(history.history['loss']) + 1))
    source = ColumnDataSource(data={'epoch': epochs, 'Train_Loss': history.history['loss'], 'Val_Loss': history.history['val_loss']})
    p = figure(title='Training and Validation Loss', x_axis_label='Epochs', y_axis_label='Loss', width=800, height=400)
    p.line('epoch', 'Train_Loss', source=source, line_width=2, color='blue', legend_label='Train Loss')
    p.line('epoch', 'Val_Loss', source=source, line_width=2, color='red', legend_label='Validation Loss')
    p.legend.location = 'top_right'
    p.legend.click_policy = 'hide'

    return p

def draw_evaluation(output_cols, y_test_inv, y_pred_inv):
    output_notebook()

    labels = [output_cols[0] + '(°C)', output_cols[1] + '(%)', output_cols[2] + '(ppm)']
    titles = ['Test Prediction:' + output_cols[0], 'Test Prediction:' + output_cols[1], 'Test Prediction:' + output_cols[2]]

    # capasity for multiple plots
    plots = []

    # horizontal axis
    x = list(range(len(y_test_inv)))

    for i in range(len(output_cols)):
        source = ColumnDataSource(data={'x': x, 'True': y_test_inv[output_cols[i]], 'Predicted': y_pred_inv[output_cols[i]]})

        # draw each plot
        p = figure(title=titles[i], width=2000, height=300, x_axis_label='Sample', y_axis_label=labels[i])
        p.line('x', 'True', source=source, line_width=2, color='blue', legend_label='True')
        p.line('x', 'Predicted', source=source, line_width=2, color='orange', legend_label='Predicted')
        p.legend.location = 'top_left'
        p.legend.click_policy = 'hide'

        # Save the plot to the list
        plots.append(p)
    
    return column(*plots)

In [3]:
# Global variables
look_back = 12
batch_size = 32
epochs = 200

input_cols = ['in_temp', 'in_humi', 'in_co2', 'out_temp', 'solar_rad', 'rain_sig1', 'hvac', 'fan', 'sprayer', 'l_win', 'r_win', 'curtain1', 'curtain2']
output_cols = ['in_temp', 'in_humi', 'in_co2']
power_cols = ['fcu_pwr', 'fan_pwr', 'l_win_pwr', 'r_win_pwr', 'curtain1_pwr', 'curtain2_pwr']

feature_ranges = {
    'in_temp': (0, 45),              # [5.4, 41.1]
    'in_humi': (0, 100),             # [29.1, 99.9]
    'in_co2': (200, 1500),           # [201, 1112]
    'out_temp': (-20, 45),           # [-15.8, 36.4]
    'wind_dir': (0, 360),            # [0, 360]
    'wind_spd': (0, 15),             # [0, 10.2]
    'solar_rad': (0, 1000),          # [0, 586]
    'cum_solar_rad': (0, 5000),      # [0, 2514]
    'dew_point': (0, 700),           # [0.3, 654.6]
    'rain_sig1': (0, 1),             # [0, 1]
    'rain_sig2': (0, 1),             # [0, 1]
    'storm_sig': (0, 1),             # [0, 1]
    'hvac': (0, 1),                  # [0, 1]
    'fan': (0, 1),                   # [0, 1]
    'sprayer': (0, 1),               # [0, 1]
    'l_win': (0, 1),                 # [0, 1]
    'r_win': (0, 1),                 # [0, 1]
    'curtain1': (0, 1),              # [0, 1]
    'curtain2': (0, 1),              # [0, 1]
}

param_grid = {
    'nb_filters': [64, 128, 256],
    'kernel_size': [2],
}

In [4]:
# 1. Load the data
data_df = pd.read_csv('final_project_data.csv', dtype={0: str})
data_df.head()

,datetime,in_temp,in_humi,in_co2,out_temp,wind_dir,wind_spd,solar_rad,cum_solar_rad,dew_point,...,curtain1,curtain2,tube_rail_pwr,fcu_pwr,fan_pwr,sprayer_pwr,l_win_pwr,r_win_pwr,curtain1_pwr,curtain2_pwr
0,1/1/2023 0:00,16.0,73.1,457,-2.2,107,1.8,0,694,11.1,...,1,1.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,1/1/2023 0:05,15.3,76.5,462,-2.2,103,1.7,0,694,11.2,...,1,1.0,0,200.0,55.0,6.3,0.0,0.0,0,0.0
2,1/1/2023 0:10,16.1,72.9,464,-2.3,111,1.8,0,694,11.2,...,1,1.0,62,200.0,54.0,5.1,0.0,0.0,0,0.0
3,1/1/2023 0:15,15.8,76.6,464,-2.3,111,1.9,0,694,11.7,...,1,1.0,16,200.0,55.0,7.5,0.0,0.0,0,0.0
4,1/1/2023 0:20,15.4,77.1,466,-2.3,115,1.6,0,694,11.4,...,1,1.0,13,200.0,54.0,5.3,0.0,0.0,0,0.0


In [5]:
# 2. Preprocess the data
X_train, y_train, X_val, y_val, X_test, y_test = data_preprocessor(data_df, input_cols, output_cols, power_cols, feature_ranges, look_back)
print(f'Training data shapes: {X_train.shape}, {y_train.shape}')
print(f'Validation data shapes: {X_val.shape}, {y_val.shape}')
print(f'Testing data shapes: {X_test.shape}, {y_test.shape}')

Training data shapes: (75677, 12, 13), (75677, 3)
Validation data shapes: (18920, 12, 13), (18920, 3)
Testing data shapes: (10511, 12, 13), (10511, 3)


In [6]:
# 3. Train the model
model, history = model_trainer(input_cols, output_cols, look_back, epochs, batch_size, X_train, y_train, X_val, y_val, param_grid)
print('Model training completed.')

[I 2025-06-17 15:06:38,883] A new study created in memory with name: no-name-21a1da00-4d67-40c3-87f8-b03d59067145


  0%|          | 0/3 [00:00<?, ?it/s]

I0000 00:00:1750140399.661187 2658772 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22283 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:41:00.0, compute capability: 8.9


x.shape= (None, 64)
model.x = (None, 12, 13)
model.y = (None, 3)
Epoch 1/200


I0000 00:00:1750140405.137707 2658917 service.cc:152] XLA service 0x7efbe802e1e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750140405.137730 2658917 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-06-17 15:06:45.302234: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1750140406.516456 2658917 cuda_dnn.cc:529] Loaded cuDNN version 90300


  47/2365 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 3.1007    

I0000 00:00:1750140412.642947 2658917 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2365/2365 ━━━━━━━━━━━━━━━━━━━━ 31s 8ms/step - loss: 0.1745 - val_loss: 0.0012 - learning_rate: 0.0020
Epoch 2/200
2365/2365 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.0013 - val_loss: 7.0110e-04 - learning_rate: 0.0020
Epoch 3/200
2365/2365 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 6.0749e-04 - val_loss: 4.6846e-04 - learning_rate: 0.0020
Epoch 4/200
2365/2365 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 3.9332e-04 - val_loss: 3.8371e-04 - learning_rate: 0.0020
Epoch 5/200
2365/2365 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 3.1594e-04 - val_loss: 2.0122e-04 - learning_rate: 0.0020
Epoch 6/200
2365/2365 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 2.7645e-04 - val_loss: 2.0062e-04 - learning_rate: 0.0020
Epoch 7/200
2365/2365 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 2.4779e-04 - val_loss: 2.9771e-04 - learning_rate: 0.0020
Epoch 8/200
2365/2365 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 2.3966e-04 - val_loss: 2.1522e-04 - learning_rate: 0.0020
Epoch 9/200
2365/2365 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step

In [7]:
# 4. Draw training history
train_history_plot = draw_traing_history(history)
show(train_history_plot)

Loading BokehJS ...

In [8]:
# 5. Evaluate the model
evaluation_plots, mae_in_temp, mae_in_humi, mae_in_co2, mse_in_temp, mse_in_humi, mse_in_co2, rmse_in_temp, rmse_in_humi, rmse_in_co2 = model_evaluator(model, X_test, y_test, output_cols, feature_ranges)
print('Model evaluation completed.')
show(evaluation_plots)
print(f"Mean Absolute Error for in_temp: {mae_in_temp:.2f}")
print(f"Mean Absolute Error for in_humi: {mae_in_humi:.2f}")
print(f"Mean Absolute Error for in_co2: {mae_in_co2:.2f}")
print(f"Mean Squared Error for in_temp: {mse_in_temp:.2f}")
print(f"Mean Squared Error for in_humi: {mse_in_humi:.2f}")
print(f"Mean Squared Error for in_co2: {mse_in_co2:.2f}")
print(f"Root Mean Squared Error for in_temp: {rmse_in_temp:.2f}")
print(f"Root Mean Squared Error for in_humi: {rmse_in_humi:.2f}")
print(f"Root Mean Squared Error for in_co2: {rmse_in_co2:.2f}")

329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


Loading BokehJS ...

Model evaluation completed.


Mean Absolute Error for in_temp: 0.16
Mean Absolute Error for in_humi: 1.18
Mean Absolute Error for in_co2: 3.84
Mean Squared Error for in_temp: 0.06
Mean Squared Error for in_humi: 2.74
Mean Squared Error for in_co2: 95.22
Root Mean Squared Error for in_temp: 0.24
Root Mean Squared Error for in_humi: 1.66
Root Mean Squared Error for in_co2: 9.76
